In [1]:
import sys
sys.path.append("..")

In [2]:
from train_autoencoder import *
from test_autoencoder import *

In [3]:
from C2D_Models import *
from PatchWise.models_PatchWise import *

In [4]:
DATA_TYPE = "HAM10000"
IMAGE_TYPE = "normal" # change
OPTIMIZER_TYPE = "adam"
LOSS_TYPE = "mse"
BATCH_SIZE = 64
DENOISING = True
IMAGE_SIZE = 128
EPOCHS = 300
EXTRA = "CMB"

In [5]:
train_data, CHANNELS = select_dataset(dataset=DATA_TYPE, isTrain=True, image_size = IMAGE_SIZE, image_type = IMAGE_TYPE)

models = [
#     C2D_AE_COMBO(channels = CHANNELS, extra_id = "default"),
    C2D_AE_COMBO(channels = CHANNELS, acb_layers=[True, True, True, False, False], sqzex_layers= [True]*5, dropouts=[0.0, 0.0, 0.0, 0.01, 0.05], extra_id = "sqzex_incacb4_dp"),
    C2D_AE_COMBO(channels = CHANNELS, acb_layers=[True, True, True, False, False], res_layers=[False, False, True, True, False], sqzex_layers= [True]*5, dropouts=[0.0, 0.0, 0.0, 0.01, 0.05], extra_id = "sqzex_incacb4_dp_res"),
]

LOSS_TYPES = [LOSS_TYPE] * len(models)
# LOSS_TYPES = ["mse", "weighted", "weighted", "weighted"]
OPTIMIZERS_TYPES = [OPTIMIZER_TYPE] * len(models)

model_files = [
        complete_model_name(
            m.__name__,
            optimizer_type=OPTIMIZER_TYPE,
            loss_type=LOSS_TYPE,
            dataset_type=DATA_TYPE,
            image_type=IMAGE_TYPE,
            isDeNoising=DENOISING,
            extra = EXTRA
        ) for m, LOSS_TYPE, OPTIMIZERS_TYPE in zip(models, LOSS_TYPES, OPTIMIZERS_TYPES)
    ]

model_files = [join_paths(["../../VAD_MODEL_TREE/"+DATA_TYPE.upper()+"/C2D/", mn]) for mn in model_files]
pprint(model_files)

['../../VAD_MODEL_TREE/HAM10000/C2D/C2D_AE_128_3X3_COMBO_SQZEX_INCACB4_DP_DENOISING_ADAM_MSE_HAM10000_NORMAL-{CMB}',
 '../../VAD_MODEL_TREE/HAM10000/C2D/C2D_AE_128_3X3_COMBO_SQZEX_INCACB4_DP_RES_DENOISING_ADAM_MSE_HAM10000_NORMAL-{CMB}']


# Training

In [6]:
optimizers = [select_optimizer[ot](m) for m,ot in zip(models, OPTIMIZERS_TYPES)]
losses = [select_loss[lt] for m,lt in zip(models, LOSS_TYPES)]

In [7]:
train_loader, val_loader = get_data_loaders(train_data, batch_size=BATCH_SIZE)

In [8]:
trainer = AutoEncoder_Trainer(
    models,
    model_files,
    train_loader, 
    val_loader,
    optimizers,
    losses,
    epochs = EPOCHS
)

In [ ]:
trainer.train()

Model: ../../VAD_MODEL_TREE/HAM10000/C2D/C2D_AE_128_3X3_COMBO_SQZEX_INCACB4_DP_DENOISING_ADAM_MSE_HAM10000_NORMAL-{CMB}/C2D_AE_128_3X3_COMBO_SQZEX_INCACB4_DP_DENOISING_ADAM_MSE_HAM10000_NORMAL-{CMB}.pth.tar
Maximum estimated Time: 7.60 hours | Will be completed by: 2021-03-24 23:58
------------------------------------------------------------

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


Wed Mar 24 16:22:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
|

In [ ]:
print("Training done")

# Testing

In [ ]:
test_data, CHANNELS = select_dataset(dataset = DATA_TYPE, image_size = IMAGE_SIZE, image_type = IMAGE_TYPE, isTrain=False, sample_stride = 1)

In [ ]:
for ae_model in trainer.autoencoder_models:
    print("-"*40)
    print(ae_model.model_file)
    ae_model.model.isTrain = False
    tester = AutoEncoder_Tester(
        model = ae_model.model,
        dataset = test_data,
        model_file = ae_model.model_file,
        stackFrames = 64,
        save_vis = False,
        useGPU = True
    )
    results = tester.test()
    print("="*40)
    del tester

In [ ]:
exit()